# Reformat Dash et al data

## Setup

In [2]:
from pathlib import Path
import pandas as pd
from pandas import isna, notna
import tidytcells
from tqdm import tqdm

In [3]:
raw_dir = Path("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/tcrdist/")
preprocessed_dir = Path(
    "/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/preprocessed/dash"
)

### Reformat and clean data

In [4]:
df = pd.read_csv(raw_dir / "dash_human.csv")

In [6]:
df = df[["v_b_gene", "cdr3_b_aa", "epitope", "count"]]
df = df.rename(
    columns={
        "v_b_gene": "TRBV",
        "cdr3_b_aa": "CDR3B",
        "epitope": "Epitope",
        "count": "duplicate_count",
    }
)

### Save reformatted data

In [8]:
df.to_csv(preprocessed_dir / "evaluation.csv", index=False)